In [1]:
# Dependencies and Setup

import json
import pandas as pd
import requests
import numpy as py
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()
from sqlalchemy import Column, Integer, String, Float, Date
import pymysql
pymysql.install_as_MySQLdb()
from sqlalchemy import create_engine, inspect, func, distinct
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
import sqlite3
from sqlite3 import Error

In [2]:
# Create engine and engine connection to SQLite Database

engine = create_engine("sqlite:///../SqLIte_files/NBAfantasyML.sqlite")
conn = engine.connect()

In [3]:
# Create function to try and connect to a sqlite database.  if it doesn't work, create one...

def create_connection(db_file):
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
    finally:
        conn.close()

In [4]:
# If connection succeeds

if __name__ == '__main__':
    create_connection("../NBAfantasyML.sqlite")

2.6.0


In [5]:
#Bring in data from internet as object then convert ot Json

schedule_response_obj=requests.get("http://data.nba.net/prod/v1/2018/schedule.json")
schedule_response=schedule_response_obj.json()

In [6]:
# Count the number of Games for the season 

games_response_count=schedule_response["league"]["standard"]
game_number=len(games_response_count)
# game_number

In [7]:
###    Elements for URL, append List, create DataFrame  ###

# Bring in Game Id and GameData ("startDataEastern") elements from NBA Schedule later used to create box_score URL
# Store elements in list
# Convert list to DataFrame and modify datatypes

# Get elements
NBA_Schedule=[]
for item in schedule_response["league"]["standard"]:
    item_dict={}
    item_dict["gameId"]=item["gameId"]
    item_dict["startDateEastern"]=item["startDateEastern"]

# Append List
    NBA_Schedule.append({"gameId":item_dict["gameId"],
                         "startDataEastern":item_dict["startDateEastern"]})

# Convert NBA_Schedule=[] list to DataFrame and modify Datatypes
NBA_Schedule_DF_initial=pd.DataFrame(NBA_Schedule)
NBA_Schedule_DF_initial["gameId"]=NBA_Schedule_DF_initial["gameId"].astype(int)
NBA_Schedule_DF_initial["startDataEastern"]=NBA_Schedule_DF_initial["startDataEastern"].astype(int)

NBA_Schedule_DF_initial.head()  

,gameId,startDataEastern
0,11800001,20180928
1,11800002,20180928
2,11800003,20180929
3,11800004,20180929
4,11800005,20180929


In [8]:
# count the number of gamesId records

game_count=len(NBA_Schedule_DF_initial["gameId"])

In [13]:
# # Generate Box_Score_URL and append to List Box_Score_URL_List

Box_score_URL_List =[{}]

# Looping through gameids 
# regular season starts October 16, 2018
#for i in range (0,game_count):
    
chosen=0

    # Generate Box Score URL using GameId and GameData ("startDataEastern")
Box_score_URL=F"http://data.nba.net/prod/v1/{NBA_Schedule_DF_initial['startDataEastern'][chosen]}/00{NBA_Schedule_DF_initial['gameId'][chosen]}_boxscore.json"

NBA_Schedule_DF_initial["boxScoreURL"]=Box_score_URL
NBA_Schedule_DF_initial["boxScoreURL"]=NBA_Schedule_DF_initial["boxScoreURL"].astype(str)
# NBA_Schedule_DF_initial.head()
print(Box_score_URL)

http://data.nba.net/prod/v1/20180928/0011800001_boxscore.json


In [14]:
# Box Score Request and request Object 

NBA_Schedule_DF_initial.boxScoreURL[chosen]
box_score_obj=requests.get(Box_score_URL)
box_score_response=box_score_obj.json()
# box_score_response

In [38]:
# Extract vteam and hteam basic box score data EXCLUDING: INDIVIDUAL PLAYER STATS 

Team_boxscores_list=[]
# Vteam
vteam_dict={}
vteam_dict["gameId"]=box_score_response["basicGameData"]["gameId"]
vteam_dict["teamId"]=box_score_response["basicGameData"]["vTeam"]["teamId"]
vteam_dict["fastBreakPoints"]=box_score_response["stats"]["vTeam"]["fastBreakPoints"]
vteam_dict["pointsInPaint"]=box_score_response["stats"]["vTeam"]["pointsInPaint"]
vteam_dict["secondChancePoints"]=box_score_response["stats"]["vTeam"]["secondChancePoints"]
vteam_dict["pointsOffTurnovers"]=box_score_response["stats"]["vTeam"]["pointsOffTurnovers"]
vteam_dict["points"]=box_score_response["stats"]["vTeam"]["totals"]["points"]
vteam_dict["fgm"]=box_score_response["stats"]["vTeam"]["totals"]["fgm"]
vteam_dict["fga"]=box_score_response["stats"]["vTeam"]["totals"]["fga"]
vteam_dict["fgp"]=box_score_response["stats"]["vTeam"]["totals"]["fgp"]
vteam_dict["ftm"]=box_score_response["stats"]["vTeam"]["totals"]["ftm"]
vteam_dict["fta"]=box_score_response["stats"]["vTeam"]["totals"]["fta"]
vteam_dict["ftp"]=box_score_response["stats"]["vTeam"]["totals"]["ftp"]
vteam_dict["tpm"]=box_score_response["stats"]["vTeam"]["totals"]["tpm"]
vteam_dict["tpa"]=box_score_response["stats"]["vTeam"]["totals"]["tpa"]
vteam_dict["tpp"]=box_score_response["stats"]["vTeam"]["totals"]["tpp"]
vteam_dict["offReb"]=box_score_response["stats"]["vTeam"]["totals"]["offReb"]
vteam_dict["defReb"]=box_score_response["stats"]["vTeam"]["totals"]["defReb"]
vteam_dict["totReb"]=box_score_response["stats"]["vTeam"]["totals"]["totReb"]
vteam_dict["assists"]=box_score_response["stats"]["vTeam"]["totals"]["assists"]
vteam_dict["pFouls"]=box_score_response["stats"]["vTeam"]["totals"]["pFouls"]
vteam_dict["steals"]=box_score_response["stats"]["vTeam"]["totals"]["steals"]
vteam_dict["turnovers"]=box_score_response["stats"]["vTeam"]["totals"]["turnovers"]
vteam_dict["blocks"]=box_score_response["stats"]["vTeam"]["totals"]["blocks"]
vteam_dict["plusMinus"]=box_score_response["stats"]["vTeam"]["totals"]["plusMinus"]
vteam_dict["min"]=box_score_response["stats"]["vTeam"]["totals"]["min"]

Team_boxscores_list.append({"gameId":vteam_dict["gameId"],
                            "teamId":vteam_dict["teamId"],
                            "fastBreakPoints":vteam_dict["fastBreakPoints"],
                            "pointsInPaint":vteam_dict["pointsInPaint"],
                            "secondChancePoints":vteam_dict["secondChancePoints"],
                            "pointsOffTurnovers":vteam_dict["pointsOffTurnovers"],
                            "points":vteam_dict["points"],
                            "fgm":vteam_dict["fgm"],
                            "fga":vteam_dict["fga"],
                            "fgp":vteam_dict["fgp"],
                            "ftm":vteam_dict["ftm"],
                            "fta":vteam_dict["fta"],
                            "ftp":vteam_dict["ftp"],
                            "tpm":vteam_dict["tpm"],
                            "tpa":vteam_dict["tpa"],
                            "tpp":vteam_dict["tpp"],
                            "offReb":vteam_dict["offReb"],
                            "defReb":vteam_dict["defReb"],
                            "totReb":vteam_dict["totReb"],
                            "assists":vteam_dict["assists"],
                            "pFouls":vteam_dict["pFouls"],
                            "steals":vteam_dict["steals"],
                            "turnovers":vteam_dict["turnovers"],
                            "blocks":vteam_dict["blocks"],
                            "plusMinus":vteam_dict["plusMinus"],
                            "min":vteam_dict["min"]})

# Hteam
hteam_dict={}
hteam_dict["gameId"]=box_score_response["basicGameData"]["gameId"]
hteam_dict["teamId"]=box_score_response["basicGameData"]["hTeam"]["teamId"]
hteam_dict["fastBreakPoints"]=box_score_response["stats"]["hTeam"]["fastBreakPoints"]
hteam_dict["pointsInPaint"]=box_score_response["stats"]["hTeam"]["pointsInPaint"]
hteam_dict["secondChancePoints"]=box_score_response["stats"]["hTeam"]["secondChancePoints"]
hteam_dict["pointsOffTurnovers"]=box_score_response["stats"]["hTeam"]["pointsOffTurnovers"]
hteam_dict["points"]=box_score_response["stats"]["hTeam"]["totals"]["points"]
hteam_dict["fgm"]=box_score_response["stats"]["hTeam"]["totals"]["fgm"]
hteam_dict["fga"]=box_score_response["stats"]["hTeam"]["totals"]["fga"]
hteam_dict["fgp"]=box_score_response["stats"]["hTeam"]["totals"]["fgp"]
hteam_dict["ftm"]=box_score_response["stats"]["hTeam"]["totals"]["ftm"]
hteam_dict["fta"]=box_score_response["stats"]["hTeam"]["totals"]["fta"]
hteam_dict["ftp"]=box_score_response["stats"]["hTeam"]["totals"]["ftp"]
hteam_dict["tpm"]=box_score_response["stats"]["hTeam"]["totals"]["tpm"]
hteam_dict["tpa"]=box_score_response["stats"]["hTeam"]["totals"]["tpa"]
hteam_dict["tpp"]=box_score_response["stats"]["hTeam"]["totals"]["tpp"]
hteam_dict["offReb"]=box_score_response["stats"]["hTeam"]["totals"]["offReb"]
hteam_dict["defReb"]=box_score_response["stats"]["hTeam"]["totals"]["defReb"]
hteam_dict["totReb"]=box_score_response["stats"]["hTeam"]["totals"]["totReb"]
hteam_dict["assists"]=box_score_response["stats"]["hTeam"]["totals"]["assists"]
hteam_dict["pFouls"]=box_score_response["stats"]["hTeam"]["totals"]["pFouls"]
hteam_dict["steals"]=box_score_response["stats"]["hTeam"]["totals"]["steals"]
hteam_dict["turnovers"]=box_score_response["stats"]["hTeam"]["totals"]["turnovers"]
hteam_dict["blocks"]=box_score_response["stats"]["hTeam"]["totals"]["blocks"]
hteam_dict["plusMinus"]=box_score_response["stats"]["hTeam"]["totals"]["plusMinus"]
hteam_dict["min"]=box_score_response["stats"]["hTeam"]["totals"]["min"]

Team_boxscores_list.append({"gameId":hteam_dict["gameId"],
                            "teamId":hteam_dict["teamId"],
                            "fastBreakPoints":hteam_dict["fastBreakPoints"],
                            "pointsInPaint":hteam_dict["pointsInPaint"],
                            "secondChancePoints":hteam_dict["secondChancePoints"],
                            "pointsOffTurnovers":hteam_dict["pointsOffTurnovers"],
                            "points":hteam_dict["points"],
                            "fgm":hteam_dict["fgm"],
                            "fga":hteam_dict["fga"],
                            "fgp":hteam_dict["fgp"],
                            "ftm":hteam_dict["ftm"],
                            "fta":hteam_dict["fta"],
                            "ftp":hteam_dict["ftp"],
                            "tpm":hteam_dict["tpm"],
                            "tpa":hteam_dict["tpa"],
                            "tpp":hteam_dict["tpp"],
                            "offReb":hteam_dict["offReb"],
                            "defReb":hteam_dict["defReb"],
                            "totReb":hteam_dict["totReb"],
                            "assists":hteam_dict["assists"],
                            "pFouls":hteam_dict["pFouls"],
                            "steals":hteam_dict["steals"],
                            "turnovers":hteam_dict["turnovers"],
                            "blocks":hteam_dict["blocks"],
                            "plusMinus":hteam_dict["plusMinus"],
                            "min":hteam_dict["min"]})


boxscore_teams_df=pd.DataFrame(Team_boxscores_list)
boxscore_teams_df

,assists,blocks,defReb,fastBreakPoints,fga,fgm,fgp,fta,ftm,ftp,...,pointsInPaint,pointsOffTurnovers,secondChancePoints,steals,teamId,totReb,tpa,tpm,tpp,turnovers
0,15,1,29,6,82,31,37.8,14,13,92.9,...,32,12,8,10,15016,37,33,9,27.3,20
1,25,9,37,12,85,41,48.2,25,16,64.0,...,60,18,17,13,1610612755,48,29,6,20.7,17


In [ ]:
# Hteam
home_team=box_score_response["basicGameData"]["hTeam"]["teamId"]

In [ ]:
# Extract ONLY player stats from box score
boxscore_player_list=[]
for boxscore_playerstats in box_score_response["stats"]["activePlayers"]:
    boxscore_player_dict={}
    boxscore_player_dict["personId"]=boxscore_playerstats["personId"]
    boxscore_player_dict["teamId"]=boxscore_playerstats["teamId"]
    boxscore_player_dict["points"]=boxscore_playerstats["points"]
    boxscore_player_dict["pos"]=boxscore_playerstats["pos"]
    boxscore_player_dict["min"]=boxscore_playerstats["min"]
    boxscore_player_dict["points"]=boxscore_playerstats["points"]
    boxscore_player_dict["fgm"]=boxscore_playerstats["fgm"]
    boxscore_player_dict["min"]=boxscore_playerstats["min"]
    boxscore_player_dict["points"]=boxscore_playerstats["points"]
    boxscore_player_dict["fgm"]=boxscore_playerstats["fgm"]
    boxscore_player_dict["fga"]=boxscore_playerstats["fga"]
    boxscore_player_dict["fgp"]=boxscore_playerstats["fgp"]
    boxscore_player_dict["ftm"]=boxscore_playerstats["ftm"]
    boxscore_player_dict["fta"]=boxscore_playerstats["fta"]
    boxscore_player_dict["ftp"]=boxscore_playerstats["ftp"]
    boxscore_player_dict["tpm"]=boxscore_playerstats["tpm"]
    boxscore_player_dict["tpa"]=boxscore_playerstats["tpa"]
    boxscore_player_dict["tpp"]=boxscore_playerstats["tpp"]
    boxscore_player_dict["offReb"]=boxscore_playerstats["offReb"]
    boxscore_player_dict["defReb"]=boxscore_playerstats["defReb"]
    boxscore_player_dict["totReb"]=boxscore_playerstats["totReb"]
    boxscore_player_dict["assists"]=boxscore_playerstats["assists"]
    boxscore_player_dict["pFouls"]=boxscore_playerstats["pFouls"]
    boxscore_player_dict["steals"]=boxscore_playerstats["steals"]
    boxscore_player_dict["turnovers"]=boxscore_playerstats["totReb"]
    boxscore_player_dict["blocks"]=boxscore_playerstats["blocks"]
    boxscore_player_dict["plusMinus"]=boxscore_playerstats["plusMinus"]
    boxscore_player_dict["dnp"]=boxscore_playerstats["dnp"]
    
    boxscore_player_list.append({"personId":boxscore_playerstats["personId"],
                                "teamId":boxscore_playerstats["teamId"],
                                "points":boxscore_playerstats["points"],
                                "pos":boxscore_playerstats["pos"],
                                "min":boxscore_playerstats["min"],
                                "points":boxscore_playerstats["points"],
                                "fgm":boxscore_playerstats["fgm"],
                                "min":boxscore_playerstats["min"],
                                "points":boxscore_playerstats["points"],
                                "fgm":boxscore_playerstats["fgm"],
                                "fga":boxscore_playerstats["fga"],
                                "fgp":boxscore_playerstats["fgp"],
                                "ftm":boxscore_playerstats["ftm"],
                                "fta":boxscore_playerstats["fta"],
                                "ftp":boxscore_playerstats["ftp"],
                                "tpm":boxscore_playerstats["tpm"],
                                "tpa":boxscore_playerstats["tpa"],
                                "tpp":boxscore_playerstats["tpp"],
                                "offReb":boxscore_playerstats["offReb"],
                                "defReb":boxscore_playerstats["defReb"],
                                "totReb":boxscore_playerstats["totReb"],
                                "assists":boxscore_playerstats["assists"],
                                "pFouls":boxscore_playerstats["pFouls"],
                                "steals":boxscore_playerstats["steals"],
                                "turnovers":boxscore_playerstats["totReb"],
                                "blocks":boxscore_playerstats["blocks"],
                                "plusMinus":boxscore_playerstats["plusMinus"],
                                "dnp":boxscore_playerstats["dnp"]})
    

    

In [ ]:
boxscore_player_df=pd.DataFrame(boxscore_player_list)
boxscore_player_df.dtypes

In [ ]:
#Load NBA_TEAMS table
# NBA_Schedule_DF_GameId.to_sql('NBA_SCHEDULE_GAME_DATE',conn, if_exists='replace', index=False)